# Import and Authenticate

**REQUIREMENTS**

You'll need to make a copy of the following - https://github.com/adugnag/gee_s1_ard 

In [8]:
# Import and Authenticate Google Earth Engine
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-jemimaofarrell')

In [9]:
# Importing libraries and preprocessing scripts

import numpy as np
import importlib

import sys
sys.path.append('../src/')
import data_processing as ppf
importlib.reload(ppf)

<module 'data_processing' from '/home/jemima/Data/NigerDelta_MangroveClassification/scripts/../src/data_processing.py'>

# Prepare SAR Data for Export

In [4]:
xmin, xmax = 6.5,7.5
ymin, ymax = 4.2,4.9
bounds = ee.Geometry.Rectangle([[xmin, ymin],[xmax, ymax]])

In [5]:
fc_db = (ee.ImageCollection('COPERNICUS/S1_GRD')
                .filterBounds(bounds)
                .sort('system:time_start')
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')))

orbit_nums = np.unique(fc_db.aggregate_array('relativeOrbitNumber_start').getInfo()) #Get orbit numbers from the region to update param_grid

In [6]:
startDate = "2016-01-01"
endDate = "2024-01-01"

# Initialise param_grid, the dates and bounds will be updated when the getS1_imcol function is called
param_grid = {  'START_DATE': startDate,
            'STOP_DATE': endDate,
            'POLARIZATION': 'VVVH',
            'ORBIT' : 'ASCENDING',
            'ORBIT_NUM': orbit_nums,
            'PLATFORM_NUMBER': 'A',
            'ROI': bounds,
            'APPLY_BORDER_NOISE_CORRECTION': True,
            'APPLY_SPECKLE_FILTERING': True,
            'SPECKLE_FILTER_FRAMEWORK':'MULTI',
            'SPECKLE_FILTER': 'GAMMA MAP',
            'SPECKLE_FILTER_KERNEL_SIZE': 9,
            'SPECKLE_FILTER_NR_OF_IMAGES':10,
            'APPLY_TERRAIN_FLATTENING': True,
            'DEM': ee.Image('USGS/SRTMGL1_003'),
            'TERRAIN_FLATTENING_MODEL': 'VOLUME',
            'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
            'FORMAT': 'DB',
            'CLIP_TO_ROI': True,
            'SAVE_ASSET': False,
            'ASSET_ID': "users/jemimaofarrell"
            }

# Example calling of getS1_imcol which returns a preprocessed annual sliced image collection for the year provided, with GLCM features appended 
start_year, end_year = 2017, 2018
s1_processed = ppf.getS1_imcol(param_grid, bounds, start_year, end_year, orbit_nums)

Number of images in collection:  114
Additional border noise correction is completed
Multi-temporal speckle filtering is completed
Radiometric terrain normalization is completed


In [7]:
s1_processed_composites = ppf.getS1_composites(param_grid, bounds, start_year, end_year, orbit_nums)
s1_processed_composites

Number of images in collection:  114
Additional border noise correction is completed
Multi-temporal speckle filtering is completed
Radiometric terrain normalization is completed


# Generate Mask if Applying to New ROI

**REQUIREMENTS**
- Download all waterbodies from Open Street Maps for most accurate masking of delta regions


In [43]:
def addNDBI(image):
  VI = image.normalizedDifference(['B12','B8']).rename('NDBI')
  return image.addBands(VI)


def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask1 = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask2 = qa.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask1).updateMask(mask2).copyProperties(image)

S2_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") 
                .filterDate('2023-08-01', '2024-01-01').filterBounds(bounds) 
                .map(addNDBI) 
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) 
                .map(maskS2clouds) 
                )

landCover = ee.ImageCollection("ESA/WorldCover/v100").first()
threshold_value = 0
NDBI_mask = S2_collection.median().select('NDBI').clip(bounds).lt(threshold_value)

In [ ]:
testImg_S1 = ppf.getS1_composites(param_grid, bounds, start_year, end_year, orbit_nums)

def reclass_function(image):
  LC_Type = image.select("Map")
  type1bins = LC_Type.where(LC_Type.eq(10), 1).where(LC_Type.eq(95), 1) # Mask everything that isn't Mangroves or Forest in the ESA landcover map
  return image.addBands(type1bins.rename('bin'))
reclassifiedMask = reclass_function(landCover)

waterways = ee.FeatureCollection("projects/ee-jemimaofarrell/assets/niger_delta/waterways_niger_delta") # Load Waterbodies from OSM, doiwnloaded as SHP in QGIS and uploaded as asset
waterMask = waterways.reduceToImage(properties=['mask'], reducer=ee.Reducer.first()).unmask(0).Not()

final_mask = reclassifiedMask.select('bin').eq(1).And(waterMask).And(NDBI_mask) # Combine into one mask

Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(testImg_S1.select('VV_median').clip(bounds), {'min':-15,'max':0}, 'Sample VV Unmasked Median',False)
Map.addLayer(testImg_S1.select('VV_median').clip(bounds).updateMask(final_mask), {'min':-15,'max':0}, 'Sample VV Median')
Map.addLayer(landCover.clip(bounds).visualize(),{},'Landcover',False)
Map.centerObject(bounds)
Map

Number of images in collection:  114
Additional border noise correction is completed
Multi-temporal speckle filtering is completed
Radiometric terrain normalization is completed


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [44]:
task = ee.batch.Export.image.toAsset(
    image=final_mask,
    description='image_export',
    assetId='projects/ee-jemimaofarrell/assets/niger_delta/static_mask',  
    region=bounds,
    scale=10,
    crs="EPSG:32632"
)
task.start()

# Export Masked Annual Composites to Drive

In [ ]:
for year in np.linspace(2016,2024,9):

  start_year = int(year)
  end_year = start_year + 1
  AnnualComp = ppf.getS1_composites(param_grid, bounds, start_year, end_year, orbit_nums)

  task = ee.batch.Export.image.toDrive(image=AnnualComp.float().updateMask(final_mask),
                                          description=str(int(year)),
                                          folder='S1 Annual Masked Tiffs', # Change to a folder name in your Drive 
                                          region=bounds,
                                          crs= 'EPSG:32632',
                                          scale= 10,
                                          maxPixels=1e13)
  task.start()